In [0]:
DECLARE OR REPLACE VARIABLE catalog_use = 'main';
DECLARE OR REPLACE VARIABLE schema_use = 'synthea';

In [0]:
SET VARIABLE catalog_use = :catalog_use;
SET VARIABLE schema_use = :schema_use; 

In [0]:
USE IDENTIFIER(catalog_use || '.' || schema_use);
SELECT current_catalog(), current_schema();

In [0]:
FROM identifier(catalog_use || '.information_schema.routines') |>
WHERE specific_schema = schema_use |>
SELECT specific_catalog, specific_schema, specific_name, routine_type, routine_definition, routine_body, data_type;

In [0]:
SELECT
  databricks_rest_get(
    endpoint => '/2.1/unity-catalog'
    ,resource => 'tables'
    ,path_parameters => NULL
    ,query_parameters => array('catalog_name=mgiglia', 'schema_name=synthea')
    ,body => NULL
  )

In [0]:
CREATE OR REPLACE FUNCTION databricks_rest_unity_catalog_get_table(
  full_name STRING COMMENT 'Full name of the table. Required path parameter.'
  ,query_parameters ARRAY<STRING> COMMENT 'An array of query parameters to apply to the endpoint URL if applicable.'
)
RETURNS VARIANT 
COMMENT 'Performs actions against the Unity Catalog Databricks REST API Endpoint.'
LANGUAGE SQL 
RETURN 
SELECT 
  databricks_rest_get(
    endpoint => '/2.1/unity-catalog'
    ,resource => 'tables'
    ,path_parameters => array(full_name)
    ,query_parameters => NULL
    ,body => NULL
  )
;

In [0]:
SELECT 
  databricks_rest_unity_catalog_get_table(
    'mgiglia.synthea.encounters'
    ,array(
      'include_delta_metadata=true'
      ,'include_browse=true'
      ,'include_manifest_capabilities=true'
    )
  ) as response


In [0]:
FROM (
  SELECT 
    databricks_rest_unity_catalog_get_table(
      'mgiglia.synthea.encounters'
      ,array(
        'include_delta_metadata=true'
        ,'include_browse=true'
        ,'include_manifest_capabilities=true'
      )
    ) as response
)
,LATERAL variant_explode(response:properties) as properties |> 
SELECT properties.key, properties.value |>
PIVOT (first(value) for key in ("spark.internal.streaming_table.internal_catalog", "spark.internal.streaming_table.internal_schema")) |>
SELECT CONCAT(`spark.internal.streaming_table.internal_catalog`, '.', `spark.internal.streaming_table.internal_schema`, '.__event_log') as event_log_table;

In [0]:
CREATE OR REPLACE FUNCTION get_event_log_table_full_name(
  full_name STRING COMMENT 'Full name of the streaming table to retrieve the event log table name full name for.'
)
RETURNS STRING
COMMENT 'Returns the full name (Unity Catalog three level namespace) of the event log table for the given streaming table.'
LANGUAGE SQL
RETURN 
FROM (
  SELECT 
    databricks_rest_unity_catalog_get_table(
      full_name
      ,array(
        'include_browse=true'
      )
    ) as response
)
,LATERAL variant_explode(response:properties) as properties |> 
SELECT properties.key, properties.value |>
PIVOT (first(value) for key in ("spark.internal.streaming_table.internal_catalog", "spark.internal.streaming_table.internal_schema")) |>
SELECT CONCAT(`spark.internal.streaming_table.internal_catalog`, '.', `spark.internal.streaming_table.internal_schema`, '.__event_log');

In [0]:
SELECT get_event_log_table_full_name('mgiglia.synthea.encounters');

In [0]:
DECLARE OR REPLACE VARIABLE event_log_table_name STRING;

SET VARIABLE event_log_table_name = get_event_log_table_full_name('mgiglia.synthea.encounters');

SELECT event_log_table_name;

In [0]:
SELECT * FROM IDENTIFIER(event_log_table_name);